As the initial step, import the required libraries that we require to perform the tasks like preprocessing, training our model and predicting the accuracy score of our model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,f1_score

Import the dataset to the simulation software and to view the first 5 rows of our dataset we used head function in pandas dataframe

In [2]:
df=pd.read_csv("/content/drive/MyDrive/aquaponics/IoTpond1.csv")
df.head()

,created_at,entry_id,Temperature (C),Turbidity(NTU),Dissolved Oxygen(g/ml),PH,Ammonia(g/ml),Nitrate(g/ml),Population,Fish_Length(cm),Fish_Weight(g)
0,2021-06-19 00:00:05 CET,1889,24.8750,100,4.505,8.43365,0.45842,193,50,7.11,2.91
1,2021-06-19 00:01:02 CET,1890,24.9375,100,6.601,8.43818,0.45842,194,50,7.11,2.91
2,2021-06-19 00:01:22 CET,1891,24.8750,100,15.797,8.42457,0.45842,192,50,7.11,2.91
3,2021-06-19 00:01:44 CET,1892,24.9375,100,5.046,8.43365,0.45842,193,50,7.11,2.91
4,2021-06-19 00:02:07 CET,1893,24.9375,100,38.407,8.40641,0.45842,192,50,7.11,2.91


In [3]:
df.describe()

,entry_id,Temperature (C),Turbidity(NTU),Dissolved Oxygen(g/ml),PH,Ammonia(g/ml),Nitrate(g/ml),Population,Fish_Length(cm),Fish_Weight(g)
count,83126.000000,83126.000000,83126.000000,83126.000000,83126.000000,8.307400e+04,83126.000000,83126.0,83124.000000,83124.00000
mean,84018.144516,24.573376,87.490160,12.390251,7.518329,2.030817e+08,458.294408,50.0,16.414686,44.56847
std,53579.484245,0.861532,25.859375,12.518253,0.534787,7.866231e+09,338.313206,0.0,5.272244,33.21549
min,1889.000000,-127.000000,1.000000,0.007000,-0.586270,6.770000e-03,45.000000,50.0,7.110000,2.91000
25%,24902.250000,24.125000,91.000000,3.440000,7.153520,4.584200e-01,146.000000,50.0,11.790000,14.19000
50%,103478.500000,24.562500,100.000000,7.133000,7.357790,6.116600e-01,347.000000,50.0,18.080000,54.70000
75%,131074.750000,24.937500,100.000000,15.819000,7.838980,1.558803e+01,823.000000,50.0,21.000000,67.52000
max,247405.000000,27.750000,100.000000,41.046000,8.551670,4.270000e+11,1936.000000,50.0,33.450000,318.64000


To check for the null values present in our dataset we use isnull() function

In [4]:
df.isnull().sum()

created_at                 0
entry_id                   0
Temperature (C)            0
Turbidity(NTU)             0
Dissolved Oxygen(g/ml)     0
PH                         0
Ammonia(g/ml)             52
Nitrate(g/ml)              0
Population                 0
Fish_Length(cm)            2
Fish_Weight(g)             2
dtype: int64

From our model we find that the fish length and fish weight are the output parameters and the rest are to be taken as input. So, the null values in the fish length and fish weight columns, their corresponding rows are being removed and for ammonia the null value rows are filled with their mean values

In [5]:
df=df[df['Fish_Weight(g)'].notna()]
df=df[df['Fish_Length(cm)'].notna()]
df=df.fillna(df.mean())

<ipython-input-5-335d705a341e>:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df=df.fillna(df.mean())


Drop all the unnecessary columns from the dataset

In [6]:
df=df.drop(columns='Population')
df=df.drop(columns='created_at')
df=df.drop(columns='entry_id')

Check the datatype of each columns in our dataset

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83124 entries, 0 to 83125
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (C)         83124 non-null  float64
 1   Turbidity(NTU)          83124 non-null  int64  
 2   Dissolved Oxygen(g/ml)  83124 non-null  float64
 3   PH                      83124 non-null  float64
 4   Ammonia(g/ml)           83124 non-null  float64
 5   Nitrate(g/ml)           83124 non-null  int64  
 6   Fish_Length(cm)         83124 non-null  float64
 7   Fish_Weight(g)          83124 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 5.7 MB


We see that our classifications labels fish_length and fish_weight are in float variables and we need to convert them to string.

In [8]:
df=df.astype({"Fish_Length(cm)":'str',"Fish_Weight(g)":'str'})

To find the corelation between the input variables

In [9]:
df.corr()

,Temperature (C),Turbidity(NTU),Dissolved Oxygen(g/ml),PH,Ammonia(g/ml),Nitrate(g/ml)
Temperature (C),1.000000,0.074850,0.098474,0.302397,0.004518,-0.372848
Turbidity(NTU),0.074850,1.000000,-0.623254,0.222368,0.012494,-0.124813
Dissolved Oxygen(g/ml),0.098474,-0.623254,1.000000,0.005328,-0.008907,-0.101527
PH,0.302397,0.222368,0.005328,1.000000,0.011411,-0.695458
Ammonia(g/ml),0.004518,0.012494,-0.008907,0.011411,1.000000,-0.024748
Nitrate(g/ml),-0.372848,-0.124813,-0.101527,-0.695458,-0.024748,1.000000


In [10]:
x=df.drop(columns=['Fish_Length(cm)','Fish_Weight(g)'])
y=df.drop(columns=['Temperature (C)','Turbidity(NTU)','Dissolved Oxygen(g/ml)','PH','Ammonia(g/ml)','Nitrate(g/ml)'])
y1=y['Fish_Length(cm)']
y2=y['Fish_Weight(g)']

Y1 corresponds to the model for fish length label and Y2 corresponds to the model for fish weight label. Splitting the data for training and testing

In [11]:
X_train,X_test,Y1_train,Y1_test=train_test_split(x,y1,test_size=0.2,random_state=0)
X_train,X_test,Y2_train,Y2_test=train_test_split(x,y2,test_size=0.2,random_state=0)

Scaling of data

In [12]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

1) Logistic regression

In [13]:
model=LogisticRegression(multi_class='multinomial')
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.00      0.00      0.00       180
       11.01       0.27      0.87      0.42       770
        11.4       0.00      0.00      0.00       454
       11.79       0.16      0.42      0.23       811
       12.09       0.00      0.00      0.00        39
       12.18       0.20      0.10      0.14       631
       14.16       0.00      0.00      0.00       121
       14.39       0.00      0.00      0.00       271
       14.62       0.00      0.00      0.00       197
       14.85       0.00      0.00      0.00       314
       15.31       0.00      0.00      0.00       137
       15.54       0.00      0.00      0.00        49
       15.73       0.00      0.00      0.00       202
       15.96       0.00      0.00      0.00       150
       16.19       0.00      0.00      0.00       163
       16.42       0.00      0.00      0.00        75
       16.65       0.00      0.00  

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.00      0.00      0.00        21
       11.37       0.00      0.00      0.00       180
       12.31       0.27      0.87      0.42       770
      121.17       0.00      0.00      0.00        18
       125.2       0.00      0.00      0.00        39
      128.55       0.00      0.00      0.00        32
       13.25       0.00      0.00      0.00       454
       131.9       0.00      0.00      0.00        65
      135.25       0.00      0.00      0.00        15
       138.6       0.00      0.00      0.00         7
       14.19       0.16      0.42      0.23       811
       145.3       0.00      0.00      0.00        14
      148.65       0.00      0.00      0.00        33
       15.13       0.20      0.10      0.14       631
       152.0       0.00      0.00      0.00        39
      198.42       0.00      0.00      0.00         4
        2.91       0.66      0.68  

2) Decision Tree Classifier

In [14]:
model=DecisionTreeClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.98      0.98      0.98       180
       11.01       0.96      0.96      0.96       770
        11.4       0.88      0.87      0.88       454
       11.79       0.92      0.92      0.92       811
       12.09       0.82      0.85      0.84        39
       12.18       0.94      0.94      0.94       631
       14.16       0.95      0.98      0.96       121
       14.39       0.91      0.89      0.90       271
       14.62       0.87      0.90      0.89       197
       14.85       0.88      0.90      0.89       314
       15.31       0.78      0.74      0.76       137
       15.54       0.93      0.82      0.87        49
       15.73       0.89      0.90      0.90       202
       15.96       0.81      0.77      0.79       150
       16.19       0.75      0.75      0.75       163
       16.42       0.81      0.87      0.84        75
       16.65       0.74      0.74  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.87      0.95      0.91        21
       11.37       0.98      0.98      0.98       180
       12.31       0.96      0.96      0.96       770
      121.17       0.83      0.83      0.83        18
       125.2       0.82      0.79      0.81        39
      128.55       0.81      0.81      0.81        32
       13.25       0.89      0.87      0.88       454
       131.9       0.79      0.83      0.81        65
      135.25       1.00      1.00      1.00        15
       138.6       0.80      0.57      0.67         7
       14.19       0.92      0.92      0.92       811
       145.3       0.71      0.86      0.77        14
      148.65       0.77      0.70      0.73        33
       15.13       0.94      0.95      0.94       631
       152.0       0.85      0.85      0.85        39
      198.42       0.50      0.50      0.50         4
        2.91       0.97      0.96  

3) Support Vector Machines

In [15]:
model=SVC()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.00      0.00      0.00       180
       11.01       0.44      0.88      0.58       770
        11.4       0.00      0.00      0.00       454
       11.79       0.39      0.55      0.45       811
       12.09       0.00      0.00      0.00        39
       12.18       0.36      0.09      0.14       631
       14.16       0.00      0.00      0.00       121
       14.39       0.14      0.72      0.24       271
       14.62       0.29      0.17      0.22       197
       14.85       0.33      0.59      0.43       314
       15.31       0.00      0.00      0.00       137
       15.54       0.00      0.00      0.00        49
       15.73       0.00      0.00      0.00       202
       15.96       0.00      0.00      0.00       150
       16.19       1.00      0.01      0.01       163
       16.42       0.00      0.00      0.00        75
       16.65       0.00      0.00  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.00      0.00      0.00        21
       11.37       0.00      0.00      0.00       180
       12.31       0.44      0.88      0.58       770
      121.17       0.00      0.00      0.00        18
       125.2       0.00      0.00      0.00        39
      128.55       0.00      0.00      0.00        32
       13.25       0.00      0.00      0.00       454
       131.9       0.00      0.00      0.00        65
      135.25       0.00      0.00      0.00        15
       138.6       0.00      0.00      0.00         7
       14.19       0.39      0.55      0.45       811
       145.3       0.00      0.00      0.00        14
      148.65       0.25      0.12      0.16        33
       15.13       0.36      0.09      0.14       631
       152.0       0.00      0.00      0.00        39
      198.42       0.00      0.00      0.00         4
        2.91       0.58      0.42  

4) Random Forest Classifier

In [16]:
model=RandomForestClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.99      0.98      0.99       180
       11.01       0.96      0.98      0.97       770
        11.4       0.91      0.89      0.90       454
       11.79       0.93      0.94      0.93       811
       12.09       0.81      0.87      0.84        39
       12.18       0.96      0.94      0.95       631
       14.16       0.99      0.98      0.99       121
       14.39       0.91      0.95      0.93       271
       14.62       0.94      0.91      0.93       197
       14.85       0.90      0.94      0.92       314
       15.31       0.84      0.78      0.81       137
       15.54       0.94      0.90      0.92        49
       15.73       0.93      0.93      0.93       202
       15.96       0.83      0.87      0.85       150
       16.19       0.83      0.80      0.82       163
       16.42       0.92      0.91      0.91        75
       16.65       0.83      0.87  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.91      0.95      0.93        21
       11.37       0.99      0.98      0.99       180
       12.31       0.96      0.98      0.97       770
      121.17       0.87      0.72      0.79        18
       125.2       0.73      0.85      0.79        39
      128.55       0.92      0.75      0.83        32
       13.25       0.91      0.89      0.90       454
       131.9       0.83      0.91      0.87        65
      135.25       1.00      1.00      1.00        15
       138.6       1.00      0.86      0.92         7
       14.19       0.93      0.94      0.93       811
       145.3       1.00      0.79      0.88        14
      148.65       0.81      0.79      0.80        33
       15.13       0.96      0.95      0.95       631
       152.0       0.83      0.90      0.86        39
      198.42       0.67      0.50      0.57         4
        2.91       0.97      0.98  

5) K Nearest Neighbours

In [17]:
model=KNeighborsClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.92      0.87      0.89       180
       11.01       0.87      0.95      0.91       770
        11.4       0.73      0.62      0.67       454
       11.79       0.76      0.81      0.79       811
       12.09       0.46      0.54      0.49        39
       12.18       0.87      0.82      0.84       631
       14.16       0.63      0.70      0.67       121
       14.39       0.58      0.65      0.61       271
       14.62       0.64      0.65      0.64       197
       14.85       0.65      0.78      0.71       314
       15.31       0.37      0.25      0.30       137
       15.54       0.50      0.31      0.38        49
       15.73       0.69      0.66      0.67       202
       15.96       0.51      0.62      0.56       150
       16.19       0.53      0.50      0.51       163
       16.42       0.77      0.55      0.64        75
       16.65       0.46      0.48  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.35      0.38      0.36        21
       11.37       0.92      0.87      0.89       180
       12.31       0.87      0.95      0.91       770
      121.17       0.39      0.39      0.39        18
       125.2       0.40      0.41      0.41        39
      128.55       0.18      0.12      0.15        32
       13.25       0.73      0.62      0.67       454
       131.9       0.44      0.65      0.53        65
      135.25       0.17      0.07      0.10        15
       138.6       0.12      0.14      0.13         7
       14.19       0.76      0.81      0.79       811
       145.3       0.43      0.43      0.43        14
      148.65       0.50      0.64      0.56        33
       15.13       0.87      0.82      0.84       631
       152.0       0.46      0.33      0.39        39
      198.42       0.50      0.25      0.33         4
        2.91       0.94      0.91  

6) Adam Boost Classifier

In [18]:
model=AdaBoostClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.00      0.00      0.00       180
       11.01       0.00      0.00      0.00       770
        11.4       0.00      0.00      0.00       454
       11.79       0.20      1.00      0.34       811
       12.09       0.00      0.00      0.00        39
       12.18       0.00      0.00      0.00       631
       14.16       0.00      0.00      0.00       121
       14.39       0.00      0.00      0.00       271
       14.62       0.00      0.00      0.00       197
       14.85       0.00      0.00      0.00       314
       15.31       0.00      0.00      0.00       137
       15.54       0.00      0.00      0.00        49
       15.73       0.00      0.00      0.00       202
       15.96       0.00      0.00      0.00       150
       16.19       0.00      0.00      0.00       163
       16.42       0.00      0.00      0.00        75
       16.65       0.00      0.00  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.00      0.00      0.00        21
       11.37       0.00      0.00      0.00       180
       12.31       0.00      0.00      0.00       770
      121.17       0.00      0.00      0.00        18
       125.2       0.00      0.00      0.00        39
      128.55       0.00      0.00      0.00        32
       13.25       0.00      0.00      0.00       454
       131.9       0.00      0.00      0.00        65
      135.25       0.00      0.00      0.00        15
       138.6       0.00      0.00      0.00         7
       14.19       0.20      1.00      0.34       811
       145.3       0.00      0.00      0.00        14
      148.65       0.00      0.00      0.00        33
       15.13       0.00      0.00      0.00       631
       152.0       0.00      0.00      0.00        39
      198.42       0.00      0.00      0.00         4
        2.91       0.00      0.00  

7) Extra Trees Classifier

In [19]:
model=ExtraTreesClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.99      0.98      0.99       180
       11.01       0.96      0.98      0.97       770
        11.4       0.91      0.89      0.90       454
       11.79       0.93      0.95      0.94       811
       12.09       0.81      0.87      0.84        39
       12.18       0.97      0.95      0.96       631
       14.16       0.99      1.00      1.00       121
       14.39       0.91      0.95      0.93       271
       14.62       0.93      0.92      0.93       197
       14.85       0.90      0.93      0.92       314
       15.31       0.86      0.72      0.79       137
       15.54       0.94      0.90      0.92        49
       15.73       0.93      0.94      0.93       202
       15.96       0.84      0.87      0.85       150
       16.19       0.82      0.82      0.82       163
       16.42       0.94      0.91      0.93        75
       16.65       0.80      0.87  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.91      1.00      0.95        21
       11.37       0.99      0.97      0.98       180
       12.31       0.96      0.98      0.97       770
      121.17       0.87      0.72      0.79        18
       125.2       0.74      0.79      0.77        39
      128.55       0.92      0.69      0.79        32
       13.25       0.91      0.88      0.90       454
       131.9       0.79      0.94      0.86        65
      135.25       1.00      1.00      1.00        15
       138.6       1.00      0.71      0.83         7
       14.19       0.93      0.94      0.93       811
       145.3       1.00      0.86      0.92        14
      148.65       0.84      0.82      0.83        33
       15.13       0.96      0.95      0.96       631
       152.0       0.80      0.85      0.83        39
      198.42       0.33      0.25      0.29         4
        2.91       0.98      0.97  

8) Gradient Boosting Classifier

In [20]:
model=GradientBoostingClassifier()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.99      0.98      0.99       180
       11.01       0.97      0.97      0.97       770
        11.4       0.89      0.89      0.89       454
       11.79       0.92      0.92      0.92       811
       12.09       0.00      0.00      0.00        39
       12.18       0.95      0.95      0.95       631
       14.16       0.97      0.97      0.97       121
       14.39       0.90      0.92      0.91       271
       14.62       0.93      0.92      0.92       197
       14.85       0.89      0.76      0.82       314
       15.31       0.79      0.71      0.75       137
       15.54       0.88      0.94      0.91        49
       15.73       0.89      0.63      0.74       202
       15.96       0.79      0.84      0.81       150
       16.19       0.70      0.76      0.73       163
       16.42       0.67      0.13      0.22        75
       16.65       0.70      0.83  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.86      0.57      0.69        21
       11.37       0.99      0.98      0.99       180
       12.31       0.97      0.97      0.97       770
      121.17       0.00      0.00      0.00        18
       125.2       0.40      0.21      0.27        39
      128.55       0.38      0.34      0.36        32
       13.25       0.89      0.89      0.89       454
       131.9       0.00      0.00      0.00        65
      135.25       0.10      0.67      0.17        15
       138.6       0.05      0.86      0.09         7
       14.19       0.92      0.92      0.92       811
       145.3       0.00      0.00      0.00        14
      148.65       0.00      0.00      0.00        33
       15.13       0.95      0.95      0.95       631
       152.0       0.00      0.00      0.00        39
      198.42       0.00      0.00      0.00         4
        2.91       0.98      0.98  

9) Naive Bayes Classifier

In [21]:
model=MultinomialNB()
data=model.fit(X_train,Y1_train)
Y1_predict=data.predict(X_test)
print("The classification report for Fish Length is:",classification_report(Y1_test,Y1_predict))
print("The confusion matrix for Fish Length is:",confusion_matrix(Y1_test,Y1_predict))
data=model.fit(X_train,Y2_train)
Y2_predict=data.predict(X_test)
print("The classification report for Fish Weight is:",classification_report(Y2_test,Y2_predict))
print("The confusion matrix for Fish Weight is:",confusion_matrix(Y2_test,Y2_predict))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Length is:               precision    recall  f1-score   support

       10.62       0.00      0.00      0.00       180
       11.01       0.13      0.56      0.22       770
        11.4       0.00      0.00      0.00       454
       11.79       0.10      0.67      0.17       811
       12.09       0.00      0.00      0.00        39
       12.18       0.00      0.00      0.00       631
       14.16       0.00      0.00      0.00       121
       14.39       0.00      0.00      0.00       271
       14.62       0.00      0.00      0.00       197
       14.85       0.00      0.00      0.00       314
       15.31       0.00      0.00      0.00       137
       15.54       0.00      0.00      0.00        49
       15.73       0.00      0.00      0.00       202
       15.96       0.00      0.00      0.00       150
       16.19       0.00      0.00      0.00       163
       16.42       0.00      0.00      0.00        75
       16.65       0.00      0.00  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The classification report for Fish Weight is:               precision    recall  f1-score   support

      101.57       0.00      0.00      0.00        21
       11.37       0.00      0.00      0.00       180
       12.31       0.13      0.56      0.22       770
      121.17       0.00      0.00      0.00        18
       125.2       0.00      0.00      0.00        39
      128.55       0.00      0.00      0.00        32
       13.25       0.00      0.00      0.00       454
       131.9       0.00      0.00      0.00        65
      135.25       0.00      0.00      0.00        15
       138.6       0.00      0.00      0.00         7
       14.19       0.10      0.67      0.17       811
       145.3       0.00      0.00      0.00        14
      148.65       0.00      0.00      0.00        33
       15.13       0.00      0.00      0.00       631
       152.0       0.00      0.00      0.00        39
      198.42       0.00      0.00      0.00         4
        2.91       0.00      0.00  

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
